In [ ]:
"""
Maybe a good next step would be to create a phylogenetic tree of tRNA sequences, and try to see 
if certain isodecoders cluster together, rather than by phylum.

Here, I will write a script to pull out (aligned) specific isotype sequences and output them to fasta format.

Next, I will download those sequences and build a phylogenetic tree in jalview. I will use that tree to see
if specific isodecoders cluster together--which would represent functionally similar isodecoders

Using my GtRNAdb-scanning framework
"""

import sys
sys.path.append('..')
sys.path.append('../sprinzl_gtrnadb/')
sys.path.append('../cca_tails/')
import pandas as pd
from nameSprinzlPositions import sprinzlAlign
import os

def parseGtRNAdbGenomes(inFile):
    """parse GtRNAdb genomes files"""
    info = {}
    chrSizes = {}
    chrKey = {}
    genKey = {}
    abbrKey = {}
    phylKey = {}
    spKey = {}
    with open(inFile, 'r') as inF:
        entries = inF.read().split('\n\n')
        
        for entry in entries[1:-1]:
            lines = entry.split('\n')
            
            abbr = lines[0].split(' ')[-1]
            acc = lines[7].split(' ')[-1]
            db = lines[4].split(' ')[-1]
            chrSizes[abbr] = {}
            chrKey[abbr] = {}
            genKey[lines[7].split(' ')[-1]] = abbr
            abbrKey[abbr] = db
            
            phylKey[db] = ' '.join(lines[11].split(' ')[5:-1])
            
            sp = ' '.join(lines[6].split(' ')[3:])
            #determine species name, handle special cases
            if sp.split(' ')[1] == 'sp.':
                spName = sp

            elif sp[0] == sp[0].upper():
                spName = ' '.join(sp.split(' ')[0:2])

            else:
                spName = sp
                
            spKey[db] = spName
                
            
            for x, y, z in zip(lines[19].split(' ')[3:], lines[21].split(' ')[3:], lines[20].split(' ')[5:]):
                if len(x.split('_')) > 1:
                    chrSizes[abbr]['_'.join(x.split('_')[1:])] = y
                else:
                    chrSizes[abbr][x] = y
                    
                chrKey[abbr][z] = x
            
            info[abbr] = acc
        inF.close()
            
    return info, chrSizes, chrKey, genKey, abbrKey, phylKey, spKey

def formatNameMap(f):
    data = pd.read_csv(f)
    
    #print(data['GTRNADB_ID'])
    data['LEGACY_ID'] = [x.replace('trna', 'tRNA') for x in data['LEGACY_ID']]
    
    nm = data[['GTRNADB_ID']].set_index(data['LEGACY_ID'] )
    
    return nm


def main():
    # The goal of this script is to find all the unique tRNA sequences in GtRNAdb, and
    # organize them by isotype, while retrieving alignments of the tRNA sequence; the
    # number of occurences of the sequence, and the assembly they occur in.
    
    acLookup = {'Ala': ['AGC', 'GGC', 'TGC', 'CGC'], 'Gly': ['ACC', 'GCC', 'TCC', 'CCC'], 
                'Pro': ['AGG', 'GGG', 'CGG', 'TGG'], 'Thr': ['AGT', 'GGT', 'CGT', 'TGT'],
                'Val': ['AAC', 'GAC', 'TAC', 'CAC'],
                'Ser': ['AGA', 'GGA', 'CGA', 'TGA', 'ACT', 'GCT'],
                'Arg': ['ACG', 'GCG', 'TCG', 'CCG', 'TCT', 'CCT'],
                'Leu': ['AAG', 'TAG', 'GAG', 'CAG', 'CAA', 'TAA'],
                'Phe': ['AAA', 'GAA'], 'Asn': ['ATT', 'GTT'],
                'Lys': ['CTT', 'TTT'], 'Asp': ['ATC', 'GTC'],
                'Glu': ['CTC', 'TTC'], 'His': ['ATG', 'GTG'],
                'Gln': ['CTG', 'TTG'], 'Ile': ['AAT', 'GAT', 'TAT'],
                'Ile2': ['CAT'], 'Met': ['CAT'], 'iMet': ['CAT'], 'fMet': ['CAT'],
                'Tyr': ['GTA', 'ATA'], 'Sup': ['TCA', 'CTA', 'TTA'],
                'Cys': ['ACA', 'GCA'], 'Trp': ['CCA'], 'SeC': ['TCA', 'CTA', 'AAA'], 'Und': ['NNN'] }
    
    domain = 'Bacteria'
    
    info, chrSizes, chrKey, genKey, abbrKey, phylKey, spKey = parseGtRNAdbGenomes('../gtrnadb_genome_info.txt')
    
    #Stores dictionary of pandas DataFrame objects which store isodecoder sequence, and associated assembly as axes
    isoacceptorCounts = {}
    for x in acLookup.keys():
        for y in acLookup[x]:
            isoacceptorCounts[x + '-' + y] = []
            
    isoacceptorCounts = pd.DataFrame(isoacceptorCounts)
    isoacceptorCounts = isoacceptorCounts[sorted(isoacceptorCounts.columns)] #Reorder columns alphabetically
    
    #Iterate through species in GtRNAdb
    for abbr in list(info.keys()):#[-175:-174]: #<--- specifically test vibrChol1 or whatever
        #keys for relevant info
        db = abbrKey[abbr]
        ph = phylKey[db]
        sp = spKey[db]
        print(db)
        isoacceptorCounts.loc[db, :] = 0
        
        namemap = pd.DataFrame()
        
        #retrieve relevant sequence information
        try:
            namemap = formatNameMap('/projects/lowelab/users/pchan/GtRNAdb2/tRNA-runs/{2}/{0}/{1}_export.csv'.format(abbr, db, domain))
            

        #Handle inconsistent naming problems (like vibrChol1)
        except FileNotFoundError:

            try:

                namemap = pd.DataFrame()

                for x in os.listdir('/projects/lowelab/users/pchan/GtRNAdb2/tRNA-runs/{1}/{0}/'.format(abbr, domain)):

                    if x.split('_')[-1] == 'export.csv':
                        namemap = formatNameMap('/projects/lowelab/users/pchan/GtRNAdb2/tRNA-runs/{2}/{0}/{1}'.format(abbr, x, domain))
                    else:
                        pass
            #Warn about issues beyond inconsistent 
            except FileNotFoundError:
                print('warning: could not access files for ' + db)
        
        
        for x in namemap.index.values:
            name = namemap.loc[x, 'GTRNADB_ID']
            
            acc = '-'.join(name.split('-')[1:3])
            
            try:
                isoacceptorCounts.loc[db, acc] += 1
            except KeyError:
                isoacceptorCounts[acc] = 0
                isoacceptorCounts.loc[db, acc] += 1
            
    isoacceptorCounts.to_csv('./gtrnadb_isoacceptor_counts.txt', sep = '\t')
        
        
    
def writeAlignmentColumns(d, file):
    with open(file, 'w') as f:
        
        f.write(','.join(d))
        f.close()
        
    
if __name__ == '__main__':
    main()
    
